1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
a = 150

print("The decimal value of", a, "is:")
print(bin(a), "in binary.")
print(hex(a), "in hexadecimal.")



The decimal value of 150 is:
0b10010110 in binary.
0x96 in hexadecimal.


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [1]:
def binaryOfPartition(partition):
    binary = str()
    while (partition):
        partition *= 2
        if (partition >= 1):
            int_part = 1
            partition -= 1
        else:
            int_part = 0
        binary += str(int_part)
    return binary
def floatingPoint(real_no):
    sign_bit = 0
    if(real_no < 0):
        sign_bit = 1
    real_no = abs(real_no)
    int_str = bin(int(real_no))[2 : ]
    partition_str = binaryOfPartition(real_no - int(real_no))
    ind = int_str.index('1')
    exp_str = bin((len(int_str) - ind - 1) + 127)[2 : ]
    mant_str = int_str[ind + 1 : ] + partition_str
    mant_str = mant_str + ('0' * (23 - len(mant_str)))
    return sign_bit, exp_str, mant_str
 

if __name__ == "__main__":
    sign_bit, exp_str, mant_str = floatingPoint(-2.250000)
 
    float_32 = str(sign_bit) + '|' + exp_str + '|' + mant_str
 
    print(float_32)

1|10000000|00100000000000000000000


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [6]:
precision = 1.0
while 1 + precision != 1:
    precision /= 2

print(1 + precision == 1)
precision *= 2
print(1 + precision == 1)
print(f'The precision found is {precision}')

True
False
The precision found is 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [4]:
import math

#5a

def myFunc1(a,b,c):
    x=(-b+math.sqrt(pow(b,2)-(4*a*c)))/(2*a)
    y=(-b-math.sqrt(pow(b,2)-(4*a*c)))/(2*a)
    return x,y
print(myFunc1(.001,1000,.001))

#5b

def myFunc2(a,b,c):
    x=((-b+math.sqrt(pow(b,2)-(4*a*c)))*(-b-math.sqrt(pow(b,2)-(4*a*c))))/((2*a)*(-b-math.sqrt(pow(b,2)-(4*a*c))))
    y=((-b-math.sqrt(pow(b,2)-(4*a*c)))*(-b+math.sqrt(pow(b,2)-(4*a*c))))/((2*a)*(-b+math.sqrt(pow(b,2)-(4*a*c))))
    return x,y
print(myFunc2(.001,1000,.001))


#5c
   
def solution_allcases(a,b,c):
    delta=(b*b)-(4*a*c)
    delta_sqrt=math.sqrt(abs((b*b)-(4*a*c)))
    if a==0: 
        print("Error: it's not a quadratic equation")
        return 
    if delta > 0:
        sol1=(-b+delta_sqrt)/(2*a) 
        sol2=(-b-delta_sqrt)/(2*a)
        print(sol1,sol2)
    if delta == 0:
        print("The two solution are the same: ", (-b)/(2*a))
    if delta < 0:
        print((-b/(2*a)), "+", (delta_sqrt/2*a),"i")
        print((-b/(2*a)), "-", (delta_sqrt/2*a),"i")
            
solution_allcases(1,3,2)

(-9.999894245993346e-07, -999999.999999)
(-9.999894245993346e-07, -999999.9999990001)
-1.0 -2.0


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
